# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<br>В качестве исходных данных получена 1 таблица с информацией о том, в каком объеме клиенты пользуются услугами оператора связи и какой у них тариф. 

<b> Поля таблиц:</b>
<br>сalls — количество звонков,
<br>minutes — суммарная длительность звонков в минутах,
<br>messages — количество sms-сообщений,
<br>mb_used — израсходованный интернет-трафик в Мб,
<br>is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


<b> Исследование разделим на несколько частей.

<b> Часть 1. Изучение общей информации:
* [1. Изучение файлов с данными, получение общей информации, загрузка библиотек.](#ch_1_1)

<b> Часть 2. Разбивка данных на выборки:
* [1. Разбивка данных на обучающую, тестовую, валидационную выборки.](#ch_2_1)
  
<b> Часть 3. Сравнительный анализ моделей:
* [1. Исследование качетсва моделей в зависимости от гиперпараметров.](#ch_3_1)

<b> Часть 4. Проверка качества моделей:
* [1. Проверка качества моделей на тестовой выборке.](#ch_4_1)

<b> Часть 5. Проверка моделей на вменяемость:
* [1. Проверка моделей на вменяемость.](#ch_5_1)
    
<b> Часть 6. Выводы:
* [1. Выводы.](#ch_6_1)

## Откройте и изучите файл <a id='ch_1_1'></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('/datasets/users_behavior.csv')
#display(df)
print(df.info())
print(df.corr())
df = df.drop('minutes', axis = 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls   minutes  messages   mb_used  is_ultra
calls     1.000000  0.982083  0.177385  0.286442  0.207122
minutes   0.982083  1.000000  0.173110  0.280967  0.206955
messages  0.177385  0.173110  1.000000  0.195721  0.203830
mb_used   0.286442  0.280967  0.195721  1.000000  0.198568
is_ultra  0.207122  0.206955  0.203830  0.198568  1.000000


## Разбейте данные на выборки <a id='ch_2_1'></a>

In [2]:
features = df.drop('is_ultra', axis = 1)
target = df['is_ultra']
features_train, features_toslpit, target_train, target_tosplit = train_test_split(features, target, test_size = 0.4, random_state = 42)
features_test, features_valid, target_test, target_valid = train_test_split(features_toslpit, target_tosplit, test_size = 0.5, random_state = 42)
print('Размер тренировочной выборки:', features_train.shape,target_train.shape)
print('Размер тестовой выборки:',features_test.shape,target_test.shape)
print('Размер валидационной выборки:',features_valid.shape,target_valid.shape)

Размер тренировочной выборки: (1928, 3) (1928,)
Размер тестовой выборки: (643, 3) (643,)
Размер валидационной выборки: (643, 3) (643,)


Разбито в пропорции 60/20/20.

## Исследуйте модели <a id='ch_3_1'></a>

Переберем значения гиперпараметров в цикле, чтобы найти наилучшую модель.

In [3]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 10):
    model_tree = DecisionTreeClassifier(random_state=42, max_depth=depth)
    model_tree.fit(features_train, target_train)
    result = model_tree.score(features_valid, target_valid)
    if result > best_tree_result:
        best_tree_model = model_tree
        best_tree_result = result
        best_tree_depth = depth
        
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_tree_result, 'Глубина дерева:',  best_tree_depth)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.8087091757387247 Глубина дерева: 3


При классификации деревом решений высочайший accuracy получился около 81%, при глубине дерева ~4~ 3.

Посмотрим на результаты для случайного леса. Сделаем перебор и по количеству деревьев, и по глубине.

In [4]:
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for est in range(1, 11):
    for depth in range(1,10):
        model_forest = RandomForestClassifier(random_state=42, n_estimators=est, max_depth = depth)
        model_forest.fit(features_train, target_train)
        result = model_forest.score(features_valid, target_valid) 
        if result > best_forest_result:
            best_forest_model = model_forest 
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print("Accuracy наилучшей модели random forest на валидационной выборке:", best_forest_result,
      '\nГлубина дерева:',  best_forest_depth,
      '\nКоличество деревьев:', est
     )

Accuracy наилучшей модели random forest на валидационной выборке: 0.8242612752721618 
Глубина дерева: 4 
Количество деревьев: 10


  У случайного леса результаты чуть лучше, но не грандиозно - разница меньше 2%.
  
  Попробуем логистическую регрессию.

In [5]:
model_regression = LogisticRegression(random_state=42)
model_regression.fit(features_train, target_train) 
result_regression = model_regression.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result_regression)

Accuracy модели логистической регрессии на валидационной выборке: 0.7527216174183515


Лучшим по результатам тестирования оказался алгоритм random forest с accuracy на валидационной выборке более 82% при 10 деревьях и глубине деревьев 4. На втором месте алгоритм дерева решений с accuracy на 1.6 процента ниже. У него глубина дерева оказалась равной 3. Ожидаемо, хуже всех себя показала логистическая регрессия, у неё accuracy чуть больше 75%.

## Проверьте модель на тестовой выборке <a id='ch_4_1'></a>

In [6]:
test_top_accuracy = model_forest.score(features_test, target_test)
print("Accuracy наилучшей модели на тестовой выборке:", test_top_accuracy)

Accuracy наилучшей модели на тестовой выборке: 0.8133748055987559


Accuracy лучшей из получившихся моделей чуть выше 81%. Рост accuracy по сравнению с валидационной выборкой в пределах 1%.

Ради интереса, проверим на тестовой выборке остальные модели.

In [7]:
test_tree_accuracy = model_tree.score(features_test, target_test)
test_regression_accuracy = model_regression.score(features_test, target_test)
print("Accuracy модели дерева решений на тестовой выборке:", test_tree_accuracy)
print("Accuracy модели логистической регрессии на тестовой выборке:", test_regression_accuracy)

Accuracy модели дерева решений на тестовой выборке: 0.7978227060653188
Accuracy модели логистической регрессии на тестовой выборке: 0.7309486780715396


На тестовой выборке модели дерева решений и логистической регрессии показали себя чуть хуже - падение accuracy 1.1% и 2% соответственно.


## (бонус) Проверьте модели на адекватность <a id='ch_5_1'></a>

С точки зрения сравнения с моделью, предсказывающей target случайно, построенные модели адекватны, ибо их accuracy существенно выше 50%. Применять их для прогноза лучше, чем прогнозировать рандомно.

In [14]:
model_dummy = DummyClassifier(random_state=42, strategy = 'most_frequent')
model_dummy.fit(features_train, target_train) 
score_valid = model_dummy.score(features_valid, target_valid)
score_test = model_dummy.score(features_test, target_test)
print("Accuracy dummy модели на валидационной выборке:", score_valid)
print("Accuracy dummy модели на тестовой выборке:", score_test)

Accuracy dummy модели на валидационной выборке: 0.6967340590979783
Accuracy dummy модели на тестовой выборке: 0.6936236391912908


## Выводы <a id='ch_6_1'></a>

Для имеющихся данных удалось построить три модели и проверить их на валидацонной выборке. Лучше всего себя показала random forest с 10 деревями и глубиной 4, имея accuracy свыше 82%. На втором месте - решающее дерево с глубиной 3. На валидационной выборке её accuracy оказался равен 80.9%. Ниже всего accuracy у модели логистической регрессии - 75.2%. Random forest на тестовой выборке имеет немного худший accuracy - 81.3%, решающее дерево  - 79.78%, модель логистической регрессии - 73.1%.